In [1]:
import timm
import torch
import timm.data
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torchvision.models as models
import torchvision.transforms as T
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"

canon_df = pd.read_csv(file_path)

canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

cols = ['ImageFilePath', 'Genus']

df = canon_df[cols]


In [3]:
df.head(2)


,ImageFilePath,Genus
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax


In [4]:


model = timm.create_model('resnext50_32x4d.a1h_in1k', pretrained=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)


for param in model.parameters():
    param.requires_grad = False


preprocess_train = timm.data.create_transform(
    input_size=(224, 224),
    is_training=True,
)

preprocess_val = timm.data.create_transform(
    input_size=(224, 224),
    is_training=False,
)

transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [5]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model.forward_features(image)
    
    return features.cpu().numpy().flatten() 


In [6]:
X = np.vstack([extract_features(img) for img in tqdm(df['ImageFilePath'])])

le = LabelEncoder()

y = le.fit_transform(df['Genus'])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 1622/1622 [32:04<00:00,  1.19s/it] 


In [7]:
models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=42)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 54.46% | Prec: 52.14% | Rec: 54.46% | F1: 53.27% | Bal Acc: 33.20% | MCC: 0.0037


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


LogisticRegression        | Acc: 71.69% | Prec: 78.44% | Rec: 71.69% | F1: 74.06% | Bal Acc: 62.74% | MCC: 0.4890


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 61.85% | Prec: 54.00% | Rec: 61.85% | F1: 55.37% | Bal Acc: 34.29% | MCC: 0.0355


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest              | Acc: 68.31% | Prec: 67.27% | Rec: 68.31% | F1: 60.42% | Bal Acc: 37.94% | MCC: 0.2295
MLP_Baseline              | Acc: 70.15% | Prec: 71.81% | Rec: 70.15% | F1: 70.70% | Bal Acc: 45.14% | MCC: 0.3783


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Agonum
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax,Mecyclothorax


In [9]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.544615,0.52143,0.544615,0.532738,0.331955,0.003681
LogisticRegression,LogisticRegression,0.716923,0.784438,0.716923,0.740636,0.627381,0.489006
SVMLinear,SVMLinear,0.8,0.771495,0.8,0.775317,0.510346,0.553742
SVMPolynomial,SVMPolynomial,0.646154,0.526011,0.646154,0.515131,0.333442,0.000202
SVMRadialBasis,SVMRadialBasis,0.652308,0.736648,0.652308,0.518198,0.336601,0.075826
NearestNeighbor,NearestNeighbor,0.618462,0.540022,0.618462,0.553749,0.342858,0.035526
RandomForest,RandomForest,0.683077,0.672676,0.683077,0.604245,0.37941,0.229497
MLP_Baseline,MLP_Baseline,0.701538,0.718087,0.701538,0.707004,0.451352,0.378335


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/46.ResNeXt-linear-probing-genus.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/46.ResNeXt-linear-probing-genus-metrics.csv", index=False)